In [1]:
import sys
import os

path = os.path.dirname(os.path.abspath("__file__"))
sys.path.insert(0, path + '/../..')

import tf_agents.policies
import tf_agents.specs
from tf_agents.environments import suite_gym, parallel_py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.metrics import tf_metrics
from tf_agents.policies import policy_saver
from tf_agents.policies.tf_py_policy import TFPyPolicy
from tf_agents.replay_buffers import tf_uniform_replay_buffer, episodic_replay_buffer
from tf_agents.drivers import dynamic_episode_driver, dynamic_step_driver
import tensorflow as tf
from tf_agents.trajectories.policy_step import PolicyStep
from policies import SavedTFPolicy
tf.config.set_visible_devices([], 'GPU')  #  allows testing during training
from tf_agents.trajectories import time_step as ts, policy_step, trajectory
from reinforcement_learning import labeling_functions
labeling_function = labeling_functions['MountainCar-v0']
from util.io.dataset_generator import map_rl_trajectory_to_vae_input
from util.io.dataset_generator import ErgodicMDPTransitionGenerator
import tensorflow_probability as tfp
import importlib

tfd = tfp.distributions

In [2]:
py_env = suite_gym.load('MountainCar-v0')
py_env.reset()
tf_env = tf_py_environment.TFPyEnvironment(py_env)
tf_env.time_step_spec()

TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=BoundedTensorSpec(shape=(2,), dtype=tf.float32, name='observation', minimum=array([-1.2 , -0.07], dtype=float32), maximum=array([0.6 , 0.07], dtype=float32)))

In [3]:
policy_dir = '../saves/MountainCar-v0/dqn_policy'

policy =  SavedTFPolicy(policy_dir)
reward_metric = tf_metrics.AverageReturnMetric()
dynamic_episode_driver.DynamicEpisodeDriver(tf_env, policy, num_episodes=5,
                                            observers=[
                                                # display_safe_labeling,
                                                lambda _: py_env.render(mode='human'),
                                                reward_metric
                                            ]).run()
reward_metric.result()


<tf.Tensor: shape=(), dtype=float32, numpy=-107.0>

In [4]:
import variational_mdp

vae_mdp = variational_mdp.load(
    '../../saves/MountainCar-v0/models/vae_LS12_MC1_ER10.0-decay=1e-05-min=0_KLA0.0-growth=5e-05_TD0.67-0.50_1e-06-2e-06_seed=20210517_PER-priority_exponent=0.99-WIS_exponent=0.4-WIS_growth_rate=7.5e-05_bucket_based_priorities_params=full_vae_optimization-relaxed_state_encoding-latent_policy/base',
    step=780000,
    discrete_action=True)

In [5]:
vae_mdp.eval_policy(eval_env=py_env, labeling_function=labeling_function, num_eval_episodes=5, render=True)

eval policy -105.2


<tf.Tensor: shape=(), dtype=float32, numpy=-105.2>